In [1]:
import sys
import os

sys.path.append(os.path.abspath("python-iso18245-master"))
import iso18245
import pandas as pd

In [2]:
print(dir(iso18245))  # This will show us all available attributes/methods

['Any', 'Dict', 'ISO_VERSION_YEAR', 'InvalidMCC', 'List', 'MCC', 'MCCNotFound', 'MCCRange', 'NamedTuple', '__all__', '__annotations__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_cached_csv', '_find_mcc_in_csv', '_load_csv', 'csv', 'files', 'get_all_mccs', 'get_all_mccs_dict', 'get_all_mccs_in_range', 'get_mcc', 'get_mcc_range', 'validate_mcc']


In [3]:
mccs = pd.DataFrame(iso18245.get_all_mccs_dict()).drop(columns=["visa_description"])
mcc_visa = pd.read_csv("mcc_visa_pdf_data.csv", dtype={"MCC": str})
mcc_visa.columns = ["visa_" + str.lower(col) for col in mcc_visa.columns]

In [4]:
mccs_merged = mccs.merge(
    mcc_visa[
        ["visa_mcc", "visa_mcc_title", "visa_mcc_description", "visa_included_in_mcc"]
    ],
    left_on="mcc",
    right_on="visa_mcc",
    how="outer",
)

In [5]:
# now merge with human labelling
mcc_human_labelling = pd.read_csv(
    "../mapping to coicop/MCC_Prometis_MAPPING.csv", sep=";", dtype={"mcc": str}
)
mcc_human_labelling.rename(
    columns={"Prometis_label": "human_assigned_coicop_label"}, inplace=True
)
mcc_human_labelling.drop(columns=["public_mcc_description"], inplace=True)
mcc_human_labelling["mcc"] = mcc_human_labelling["mcc"].str.zfill(4)


In [6]:
mcc_human_labelling

,mcc,human_assigned_coicop_label
0,0742,CULTURAL AND RECREATIONAL SERVICES
1,0744,"ALCOHOLIC BEVERAGES, TOBACCO AND NARCOTICS"
2,0763,GARDEN PRODUCTS AND PETS
3,0780,"SERVICES FOR MAINTENANCE, REPAIR AND SECURITY ..."
4,1520,"SERVICES FOR MAINTENANCE, REPAIR AND SECURITY ..."
...,...,...
1000,3826,ACCOMMODATION SERVICES
1001,3807,ACCOMMODATION SERVICES
1002,3791,ACCOMMODATION SERVICES
1003,3832,ACCOMMODATION SERVICES


In [7]:
mccs_merged = mccs_merged.merge(
    mcc_human_labelling, left_on="mcc", right_on="mcc", how="outer"
)
mccs_merged

,mcc,mcc_range_start,mcc_range_end,mcc_range_description,mcc_range_reserved_flag,iso_description,usda_description,stripe_description,stripe_code,visa_req_clearing_name,alipay_description,mastercard_description,mastercard_abbreviated_airline_name,amex_description,visa_mcc,visa_mcc_title,visa_mcc_description,visa_included_in_mcc,human_assigned_coicop_label
0,0742,0700,0999,Agricultural services,False,Veterinary services,Veterinary Services,Veterinary Services,veterinary_services,,Veterinary services,Veterinary Services,,Veterinary Services,0742,Veterinary Services,Merchants classified with this MCC are license...,"Animal Doctors, Hospitals Clinics – Pet Hospit...",CULTURAL AND RECREATIONAL SERVICES
1,0743,0700,0999,Agricultural services,False,Wine producers,,,,,Wine producers,,,Wine Producers,NaN,NaN,NaN,NaN,"ALCOHOLIC BEVERAGES, TOBACCO AND NARCOTICS"
2,0744,0700,0999,Agricultural services,False,Champagne producers,,,,,Champagne producers,,,Champagne Producers,NaN,NaN,NaN,NaN,"ALCOHOLIC BEVERAGES, TOBACCO AND NARCOTICS"
3,0763,0700,0999,Agricultural services,False,Agricultural co-operatives,Agricultural Co-operatives,Agricultural Cooperative,agricultural_cooperative,,,Agricultural Cooperatives,,Agricultural Cooperatives,0763,Agricultural Co-operatives,Merchants classified with this MCC provide far...,Farm Management Services Orchards Vineyards,GARDEN PRODUCTS AND PETS
4,0780,0700,0999,Agricultural services,False,Landscaping and horticultural services,Horticultural Services,Landscaping Services,landscaping_services,,Landscaping and horticultural services,Horticultural and Landscaping Services,,Landscaping and Horticultural Services,0780,Landscaping and Horticultural Services,Merchants classified with this MCC provide lan...,Gardening Services Horticulture Services Servi...,"SERVICES FOR MAINTENANCE, REPAIR AND SECURITY ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,9701,9403,9999,Other,False,,Visa Credential Service ( For Visa Only),,,,,,,,NaN,NaN,NaN,NaN,SERVICES - OTHER
1054,9702,9403,9999,Other,False,,GCAS Emergency Services ( For Visa Only),,,,,,,,9702,Emergency Services (GCAS) (Visa use only),NaN,NaN,SERVICES - OTHER
1055,9950,9403,9999,Other,False,,Intra - Company Purchases ( For Visa Only),Intra-Company Purchases,intra_company_purchases,,,,,Internal Sales - Corporate,9950,Intra-Company Purchases,Merchants classified with this MCC process pur...,NaN,SERVICES - OTHER
1056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3168,HAINAN AIRLINES,NaN,NaN,NaN


In [8]:
dict(mccs_merged.iloc[700:710].iloc[2])

{'mcc': '3789',
 'mcc_range_start': '3000',
 'mcc_range_end': '3999',
 'mcc_range_description': 'Reserved for private use',
 'mcc_range_reserved_flag': True,
 'iso_description': '',
 'usda_description': '',
 'stripe_description': '',
 'stripe_code': '',
 'visa_req_clearing_name': '',
 'alipay_description': '',
 'mastercard_description': "Smugglers' Notch Resort",
 'mastercard_abbreviated_airline_name': '',
 'amex_description': "SMUGGLER'S NOTCH RESORT",
 'visa_mcc': '3789',
 'visa_mcc_title': 'SMUGGLER’S NOTCH RESORT',
 'visa_mcc_description': nan,
 'visa_included_in_mcc': nan,
 'human_assigned_coicop_label': 'ACCOMMODATION SERVICES'}

In [9]:
mccs_merged.to_csv("mccs_merged_all_sources.csv", index=False)

In [ ]:
mccs_merged